In [9]:
from dotenv import load_dotenv
load_dotenv()

import os

RIOT_API_KEY = os.getenv('RIOT_API_KEY')

In [10]:
import requests
from typing import TypedDict, List
import time

class SummonerData(TypedDict):
    puuid: str
    gameName: str
    tagLine: str

class Metadata(TypedDict):
    dataVersion: str
    matchId: str
    participants: List[str]

class PerksStyleSelection(TypedDict):
    perk: int
    var1: int
    var2: int
    var3: int

class PerksStyle(TypedDict):
    description: str
    selections: List[PerksStyleSelection]

class Perks(TypedDict):
    statPerks: dict
    styles: List[PerksStyle]

class Participant(TypedDict):
    participantId: int
    puuid: str
    summonerName: str
    championName: str
    teamId: int
    spell1Id: int
    spell2Id: int
    stats: dict
    perks: Perks

class TeamObjective(TypedDict):
    first: bool
    kills: int

class TeamObjectives(TypedDict):
    baron: TeamObjective
    dragon: TeamObjective
    inhibitor: TeamObjective
    riftHerald: TeamObjective
    tower: TeamObjective

class Team(TypedDict):
    teamId: int
    win: bool
    objectives: TeamObjectives

class Info(TypedDict):
    gameId: int
    gameDuration: int
    gameMode: str
    gameType: str
    teams: List[Team]
    participants: List[Participant]

class MatchResponse(TypedDict):
    metadata: Metadata
    info: Info

class RiotClient:
    def __init__(self, api_key: str):
        self.api_key = api_key

    def riot_api_request(self, url: str):
        headers = {
            'X-Riot-Token': self.api_key
        }
        response = requests.get(url, headers=headers)
        return response.json()
    
    def get_summoner_by_name(self, name: str, tag: str) -> SummonerData:
        url = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{name}/{tag}'
        return self.riot_api_request(url)
    
    def get_latest_game_ids(self, puuid: str):
        timestamp_of_2_hours_ago = int(time.time()) - 2 * 60 * 60
        url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=3&startTime={timestamp_of_2_hours_ago}'
        return self.riot_api_request(url)
    
    def get_current_game(self, game_id: str) -> MatchResponse:
        url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{game_id}'
        return self.riot_api_request(url)


riotClient = RiotClient(RIOT_API_KEY)
username = 'CRIA DE IPANEMA'
tag = 'PIT'
summoner = riotClient.get_summoner_by_name(username, tag)
print(summoner)
game_ids = riotClient.get_latest_game_ids(summoner['puuid'])
print(game_ids)
current_game_match = riotClient.get_current_game(game_ids[0])
player_team = 1 if [participant for participant in current_game_match['info']['participants'] if participant['puuid'] == summoner['puuid']][0]['teamId'] == 100 else 2

import json
# with open('game_match.json', 'w') as f:
#     json.dump(riotClient.get_current_game('NA1_5150433523'), f)


# puuid = 'U8A4p4luR8eSiCwn-GGaw75eTxCageiY19_YuypDvH11cj2CoNKPBk5bqq1RXbSYmU3HCln-hoy86g'
# game_id = 'NA1_5150433523'
with open('game_match.json', 'r') as f:
    current_game_match = json.load(f)


# dataset from kaggle
# https://www.kaggle.com/datasets/gyejr95/league-of-legendslol-ranked-games-2020-ver1?select=match_loser_data_version1.csv
# curl -L -o ./data/archive.zip https://www.kaggle.com/api/v1/datasets/download/gyejr95/league-of-legendslol-ranked-games-2020-ver1
# unzip data/archive.zip data



{'puuid': '95I-BpKxPhqZTwcbUVSw1oYDNRvxdF7xN8AdlS-k3WZhnejiPNV_OBC480HQhLS0rYwMrEsPexsjUw', 'gameName': 'CRIA DE IPANEMA', 'tagLine': 'PIT'}
['BR1_3021347173', 'BR1_3021318833']


In [11]:
import pandas as pd

# winners team stats csv has the following columns
# 'id', 'teamId', 'win', 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 'towerKills', 'inhibitorKills', 'baronKills', 'dragonKills', 'vilemawKills', 'riftHeraldKills', 'dominionVictoryScore', 'bans', 'gameId'
winners = pd.read_csv('data/match_winner_data_version1.csv')

# losers team stats csv has the following columns
# 'id', 'teamId', 'win', 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 'towerKills', 'inhibitorKills', 'baronKills', 'dragonKills', 'vilemawKills', 'riftHeraldKills', 'dominionVictoryScore', 'bans', 'gameId'
losers = pd.read_csv('data/match_loser_data_version1.csv')

# The both csvs are equals, just the 'win' column is 'Win' or 'Fail'
# I will merge both csvs into one now
match_data = pd.concat([winners, losers])

# fine, now I will train a model to predict the winner team based on current match data
# I will use the following columns to train the model
# 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 'towerKills', 'inhibitorKills', 'baronKills', 'dragonKills', 'riftHeraldKills'
# and the target column will be 'win'
# must convert data to numeric
match_data['win'] = match_data['win'].apply(lambda x: True if x == 'Win' else False)
match_data['firstBlood'] = match_data['firstBlood'].apply(lambda x: int(x))
match_data['firstTower'] = match_data['firstTower'].apply(lambda x: int(x))
match_data['firstInhibitor'] = match_data['firstInhibitor'].apply(lambda x: int(x))
match_data['firstBaron'] = match_data['firstBaron'].apply(lambda x: int(x))
match_data['firstDragon'] = match_data['firstDragon'].apply(lambda x: int(x))
match_data['firstRiftHerald'] = match_data['firstRiftHerald'].apply(lambda x: int(x))
match_data['towerKills'] = match_data['firstRiftHerald'].apply(lambda x: int(x))
match_data['inhibitorKills'] = match_data['firstRiftHerald'].apply(lambda x: int(x))
match_data['baronKills'] = match_data['firstRiftHerald'].apply(lambda x: int(x))
match_data['dragonKills'] = match_data['firstRiftHerald'].apply(lambda x: int(x))
match_data['riftHeraldKills'] = match_data['firstRiftHerald'].apply(lambda x: int(x))
match_data['riftHeraldKills'] = match_data['firstRiftHerald'].apply(lambda x: int(x))

x = match_data[['firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 'towerKills', 'inhibitorKills', 'baronKills', 'dragonKills', 'riftHeraldKills']]
y = match_data['win']





In [12]:

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

model = RandomForestClassifier()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy*100}%')

# now I will predict the winner team of the current game
# I will use the current_game_match to predict the winner team
# I will use the following columns to predict the winner team
# 'firstBlood', 'firstTower', 'firstInhibitor', 'firstBaron', 'firstDragon', 'firstRiftHerald', 'towerKills', 'inhibitorKills', 'baronKills', 'dragonKills', 'riftHeraldKills'
# and the target column will be 'win'
current_game_match_info = current_game_match['info']
current_game_match_teams = current_game_match_info['teams']
current_game_match_team1 = current_game_match_teams[0]
current_game_match_team2 = current_game_match_teams[1]


# first blood is value inside each participant, I must get the team who have the first blood
participant_who_take_first_blood = [participant for participant in current_game_match_info['participants'] if participant['firstBloodKill'] == True][0]

current_game_match_team1_data = {
    'firstBlood': participant_who_take_first_blood['teamId'] == 100,
    'firstTower': current_game_match_team1['objectives']['tower']['first'],
    'firstInhibitor': current_game_match_team1['objectives']['inhibitor']['first'],
    'firstBaron': current_game_match_team1['objectives']['baron']['first'],
    'firstDragon': current_game_match_team1['objectives']['dragon']['first'],
    'firstRiftHerald': current_game_match_team1['objectives']['riftHerald']['first'],
    'towerKills': current_game_match_team1['objectives']['tower']['kills'],
    'inhibitorKills': current_game_match_team1['objectives']['inhibitor']['kills'],
    'baronKills': current_game_match_team1['objectives']['baron']['kills'],
    'dragonKills': current_game_match_team1['objectives']['dragon']['kills'],
    'riftHeraldKills': current_game_match_team1['objectives']['riftHerald']['kills']
}

current_game_match_team2_data = {
    'firstBlood': participant_who_take_first_blood['teamId'] == 200,
    'firstTower': current_game_match_team2['objectives']['tower']['first'],
    'firstInhibitor': current_game_match_team2['objectives']['inhibitor']['first'],
    'firstBaron': current_game_match_team2['objectives']['baron']['first'],
    'firstDragon': current_game_match_team2['objectives']['dragon']['first'],
    'firstRiftHerald': current_game_match_team2['objectives']['riftHerald']['first'],
    'towerKills': current_game_match_team2['objectives']['tower']['kills'],
    'inhibitorKills': current_game_match_team2['objectives']['inhibitor']['kills'],
    'baronKills': current_game_match_team2['objectives']['baron']['kills'],
    'dragonKills': current_game_match_team2['objectives']['dragon']['kills'],
    'riftHeraldKills': current_game_match_team2['objectives']['riftHerald']['kills']
}

current_game_match_team1_data = pd.DataFrame([current_game_match_team1_data])
current_game_match_team2_data = pd.DataFrame([current_game_match_team2_data])

# # I want to predict showing the probability of each team to win
winner_team_probability = model.predict_proba(current_game_match_team1_data)
print('---------------------------------------------------------------')
print(f'A chance do time 1 ganhar é de {winner_team_probability[0][1]*100}% calculada utilizando os dados do primeiro time')

loser_team_probability = model.predict_proba(current_game_match_team2_data)
print('---------------------------------------------------------------')
print(f'A chance do time 1 ganhar é de {loser_team_probability[0][0]*100}% calculada utilizando os dados do segundo time')

print('---------------------------------------------------------------')
print(f'O time vencedor pela API da Riot foi o time {1 if current_game_match_team1["win"] else 2}, sendo que nosso jogador está no time {player_team}')

Model Accuracy: 84.12317375723606%
---------------------------------------------------------------
A chance do time 1 ganhar é de 92.70714529861415% calculada utilizando os dados do primeiro time
---------------------------------------------------------------
A chance do time 1 ganhar é de 84.05334732855722% calculada utilizando os dados do segundo time
---------------------------------------------------------------
O time vencedor pela API da Riot foi o time 1, sendo que nosso jogador está no time 2
